In [15]:
# 라이브러리
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
path = "C:/Users/gihun/To_raise_a_child_well/data/gihun/dataset/"

In [17]:
sum_df = pd.read_excel(path+'전국데이터프레임2006~2021.xlsx',index_col=0)
sum_df

,실업률,이혼율,고령화비율,혼인건수,소비자물가지수,출산아수/가임기여성의수,아파트전세지수,미분양주택지수,은행예금지수,경제활동참가율,지가변동률
시점,,,,,,,,,,,
1999-06-01,5.6,0.283282,NaN,36243.0,61.349,NaN,NaN,NaN,NaN,49.0,NaN
1999-07-01,5.6,0.402293,NaN,25464.0,61.140,NaN,NaN,NaN,NaN,48.7,NaN
1999-08-01,5.0,0.548160,NaN,19217.0,61.765,NaN,NaN,NaN,NaN,48.1,NaN
1999-09-01,4.2,0.549469,NaN,16485.0,61.921,NaN,NaN,NaN,NaN,49.2,NaN
1999-10-01,4.0,0.412334,NaN,24713.0,62.389,NaN,NaN,NaN,NaN,49.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,2.5,NaN,17.8,NaN,108.930,0.001486,102.8,102.8,1958348.3,55.1,NaN
2022-10-01,2.4,NaN,17.9,NaN,109.210,0.001486,101.4,101.4,1967290.0,55.0,NaN
2022-11-01,2.3,NaN,17.9,NaN,109.100,0.001486,99.0,99.0,1970661.1,55.1,NaN


In [18]:
# 독립변수,종속변수 설정
X = sum_df.drop(columns='출산아수/가임기여성의수').loc['2006-01-01':'2021-12-01'].values
y = sum_df['출산아수/가임기여성의수'].loc['2006-01-01':'2021-12-01'].values
X.shape, y.shape

((192, 10), (192,))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
# 파라미터설정
param_grid = {'max_depth': [3, 4, 5, 6],'learning_rate': [0.1, 0.01, 0.001],'n_estimators': [50, 100, 200], 'subsample':[0.5,0.8,1.0], 'eta':[0.2,0.25,0.3]}
# max_depth: 나무의 최대 깊이, 이 값을 늘리면 과적합 생길수도 있음 default 6
# min_child_weight: child_node에 필요한 인스턴스 가중치의 최소합계, 이 값이 클 수록 보수적이고 편향이 증가하게 돼서 변동성이 작아짐 default 1
# gamma: 감마가 클수록 알고리즘이 보수적임 default 0
# subsample: 0.5로 설정하면 xgboost가 나무를 성장시키기 전에 훈련데이터의 절반을 무작위로 샘플링한다. 이것은 과적합을 방지할 수도 있다 default 1
# eta: 과적합을 방지하기 위해 업데이트에 사용되는 단계 크기 축소입니다. default 0.3 - lgb에서 0.2는 무시한다고함

# 모델설정
lgb = LGBMRegressor()

# 그리드서치 파라미터 생성
grid = GridSearchCV(estimator=lgb, param_grid=param_grid, cv=5)
grid.fit(X_train_scaled, y_train)
# 출력
print("Best hyperparameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)

# 제일 좋은 것으로 다시돌리기
best_lgb = LGBMRegressor(max_depth=grid.best_params_['max_depth'],
                        learning_rate=grid.best_params_['learning_rate'],
                        n_estimators=grid.best_params_['n_estimators'])
best_lgb.fit(X_train_scaled, y_train)

# 평가하기 
train_score = best_lgb.score(X_train_scaled, y_train)
print("Train set score: ", train_score)
test_score = best_lgb.score(X_test_scaled, y_test)
print("Test set score: ", test_score)

[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.2 

In [21]:
# 출력
print("Best hyperparameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)
# 평가하기 
train_score = best_lgb.score(X_train_scaled, y_train)
print("Train set score: ", train_score)
test_score = best_lgb.score(X_test_scaled, y_test)
print("Test set score: ", test_score)

Best hyperparameters:  {'eta': 0.2, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.5}
Best score:  0.7878763371766739
Train set score:  0.9810983535682666
Test set score:  0.9480011254400686


In [22]:
best_lgb.feature_importances_.sum()

1019